### 1 Parse Bank
<br>

In [1]:
import pandas as pd
import os, re
from pdfminer.high_level import extract_pages, extract_text

%run ./_projectDoc.ipynb

In [2]:
os.chdir('/Users/lorenkarish/Desktop/JBS/Bank/')

files = ['08-2022.pdf','09-2022.pdf','10-2022.pdf','11-2022.pdf','12-2022.pdf']

log, history, end_page = {},{},{}
f_dates, f_memo, f_dollars = {},{},{}

exceptions = ['Average Available Balance', 'Post Date', 'Amount', 'Debits', 'CORPORATION',
              'CORPORATIO', 'Credits', 'Balance','Description', 'Checking Account Statements',
              '353B2A2A64DD174EA02A1BB8F7C8F6C0', ' ','JBS PIPELINE LLC', 'Checks Cleared',
              'Check Nbr', 'Date','20220831', '20220931','20221031','20221131','20221231',
              'TX_3700_SAUNDERS CONSTRUCTION, IN_CASTLE','ROCK INDUST']

SEPT_WTH = ['$385,500.00','$60,000.00','$354,853.61', '$673.80','$276,500.00','$110,500.00',
            '$0.05','$0.37','$1,000.00','$1,292.26', '$1,860.00','$13,382.75','$19,933.20','$431.64',
            '$1,379.40','$20,000.00','$113,500.00','$24,160.17', '$10,961.39','$24,852.20','$132,758.32',
            '$109,000.00','$179,064.10','$157,000.00', '$20,000.00','$96,000.00','$43,552.57', '$568,187.07']

for f in files:
    
    mode = "off"
    page = 1
    pg_stop = 50
    dates, memo, dollars = [],[],[]
    
    text = extract_text(f)
    key = f[:2]
     
    for line in text.splitlines():
        
        if line == "Post Date":
            mode = "stream" 
        
        if line.startswith('XXXXXXXXXXXXX3669 Statement'):
            mode = "off"
            Process_Page(dates, memo, dollars, page)
            dates, memo, dollars = [],[],[]
            page += 1
            
        if line == "Checks Cleared":
            pg_stop = page
       
        if page > pg_stop:
            break
        else:    
            if mode == "stream":
                if ((len(line) > 0) & (line not in exceptions)):
                    Process_Data(line) 
                    
    df = pd.concat([v for k,v in log.items()])
    history[key] = df
    log = {}

<br>

### Monthly Activity
 -  parsed data (except for the final page of activity) stored in 'history' 
 -  convert Balance type (text to numeric)
 -  calculate transaction Amount (b/c parser only captures Balance for each row)

In [3]:
history.keys()

dict_keys(['08', '09', '10', '11', '12'])

In [4]:
for k,v in history.items():
    v['Balance'] = v['Balance'].replace('[\$,]', '', regex=True).astype(float)
    v.insert(2, "Amount", v['Balance'].diff().fillna(0))

<br>

### Final Page 

 Several issues w/ reading dollar amounts:
  - irregular line sequence 
  - presence of non-essential data (checks cleared, daily balances, etc. 
  - Sept PDF has unique issues

In [5]:
for k,v in f_dates.items():
    if k!= '12':
        line_stop = f_memo[k].index("Ending Balance") + 1
        f_memo[k] = f_memo[k][:line_stop]
        
for k,v in f_memo.items():
    if k!= '12':
        line_stop = f_memo[k].index("Ending Balance") + 1
        f_dates[k] = f_dates[k][:line_stop]
               
for k in f_dates.keys():
    if k!= '12':
        final_page = list(zip(f_dates[k],f_memo[k]))
        df = pd.DataFrame(final_page, columns=['Date', 'Desc'])
        end_page[k] = df                  

<br>

Because of above issues some data on the last page had to be hard-coded:

 -  convert Balance from text to numeric
 -  calc line Amount

In [6]:
data = {'08': ['-$228,876.94', '-$228,977.70', '-$229,195.46', '-$229,867.16', '-$231,009.69',
       '-$232,292.19', '-$233,918.29', '-$236,043.29', '-$241,322.45', '-$312,287.45',
       '$212.55','$404.71','$348.33','$250.97', '$82.31',  '-$471.42','-$1,887.58', 
       '-$3,853.49','-$7,595.87', '-$59,546.87','$453.13','$11,619.88','$11,364.88',
       '$10,284.88','$6,621.84','$613.44','-$23,241.56','-$51,021.66','-$94,301.43',
       '-$159,793.31','-$248,790.21', '-$165,790.21','-$120,790.21','-$120,810.21',
       '-$120,855.21','-$120,862.21', '-$120,862.21'],
        
        '09': ['$973.04', '$973.04'],
        
        '10': ['-$99,072.80', '$427.20','$110,811.88','$55,811.88', '$911.88', '-$55,538.24', 
               '-$81,459.07', '-$84,523.26', '-$95,715.93', '-$96,515.93', '-$166,871.22', 
               '-$166,916.22','$583.78','$583.78'],
        
        '11': ['$623,350.63', '$600,907.38', '$488,052.24', '$369,695.07', '$288,695.07',
       '$232,244.95', '$206,324.12', '$202,406.39', '$201,870.99', '$132,955.71',
       '$132,910.71', '$132,910.71']}

for k,v in end_page.items():
    v.insert(2, "Balance", pd.Series(data[k]))
    v['Balance'] = v['Balance'].replace('[\$,]', '', regex=True).astype(float)
    v.insert(2, "Amount", v['Balance'].diff().fillna(0))

<br>
combine individual pages into full month:

In [7]:
for k in history.keys():
    if k!= '12':
         history[k] = pd.concat([history[k],end_page[k]])

<br>
combine months into one dataframe:

In [8]:
bank = pd.concat([v for k,v in history.items()])

<br>

- define Date as a datetime object
- add Month column   

In [9]:
bank['Date'] = pd.to_datetime(bank['Date'])
bank.insert(0, "Month", bank['Date'].dt.strftime('%b'))
bank = bank.reset_index(drop=True)

<br>

- handle problem rows 
- set Month to categorical variable
- generate .txt output file 
- store data for later use

In [10]:
bank.iat[149,3] = -100000
bank.iat[284,3] = -521500
bank.iat[338,3] = -10000
bank.iat[470,3] = -10000
bank.iat[0,0] = 'Aug'

bank['Month'] = pd.Categorical(bank['Month'],['Aug','Sep','Oct','Nov','Dec'])

bank.to_csv('parsed_Aug-Dec.txt',index=False)  

%store bank

Stored 'bank' (DataFrame)
